**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` takes as input the state $s$ and returns the index (from 0 to 3) of the corresponding action that it chooses to take. If the label `train` is set as True, the function will generate a (pesudo-)random in the interval $[0,1]$ and compare it with a threshold `epsilon` in order to decide whether it should take a random action or act via the policy of the agent. Thus, the threshold `epsilon` corresponds to the probability of choosing a random action, i.e. _exploring_, which is essential to discover new actions that may help increasing the final value. On the contrary, 1-`epsilon` corresponds to the probability of _exploiting_, i.e. choosing the action that will lead to known rewards. If the label `train` is set as False, the agent will always act via its policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the mouse
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        # MP4 is not supported by my broser, so I save as ogg as well
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        skvideo.io.vwrite(str(e) + '.ogg', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0,0] = 256
        b[self.board < 0,2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.x,self.y] = 0
        
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [19]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50 # set small when debugging
epochs_test = 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/ogg;base64,{0}" type="video/ogg" />
             </video>'''.format(encoded.decode('ascii'))
    
    # MP4 is not supported by my browser, so I display an OGG version
    """
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))
     """

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Both of them are $2 \times 2$ arrays, i.e. matrices.

`position` is updated at each action: it is initialised with 0's; filled with the value -1 on the edges (positions that cannot be reached); and with a single value 1 representing the current position of the agent.

`board` is used to represent where the rewards (bonus) and penalisations (malus) are placed in this two-dimensional environment. Note that `board` is defined in the method `reset` by placing bonus and malus, both of which are randomly generated according to a binomial distribution. In addition, in the method `act`, the bonus or malus placed on the current position of the agent is collected, by accessing the correspoding position of the array `board`.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [20]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Chosses an action randomly
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [21]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [22]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
# HTML(display_videos('random0.mp4'))
HTML(display_videos('random0.ogg'))

Win/lose count 11.0/20.0. Average score (-9.0)
Win/lose count 11.0/17.0. Average score (-7.5)
Win/lose count 12.0/11.0. Average score (-4.666666666666667)
Win/lose count 8.0/9.0. Average score (-3.75)
Win/lose count 8.5/11.0. Average score (-3.5)
Win/lose count 5.0/9.0. Average score (-3.5833333333333335)
Win/lose count 10.0/15.0. Average score (-3.7857142857142856)
Win/lose count 10.0/15.0. Average score (-3.9375)
Win/lose count 9.5/16.0. Average score (-4.222222222222222)
Win/lose count 9.5/16.0. Average score (-4.45)
Win/lose count 9.5/14.0. Average score (-4.454545454545454)
Win/lose count 8.5/17.0. Average score (-4.791666666666667)
Win/lose count 13.5/13.0. Average score (-4.384615384615385)
Win/lose count 6.5/8.0. Average score (-4.178571428571429)
Win/lose count 14.5/16.0. Average score (-4.0)
Win/lose count 8.0/10.0. Average score (-3.875)
Win/lose count 5.5/8.0. Average score (-3.7941176470588234)
Win/lose count 10.5/18.0. Average score (-4.0)
Win/lose count 10.5/12.0. Averag

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__

1. Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

2. Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

3. Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a';\theta)-Q(s,a;\theta)\Vert^{2}.
\end{equation*}




1. To prove that $Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$ we apply the properties of Markov decision processes (MDP) : 
$$
\begin{align}
Q^{\pi}(s,a) &= E\left[ \sum_{t=0}^{T} \gamma^t r(s_t,a_t) | s_0=s, a_0=a \right]\\
&= r(s,a) + E\left[ \sum_{t=1}^{T} \gamma^t r(s_t,a_t) | s_0=s, a_0=a \right]\\
&= r(s,a) + \gamma\sum_{(s',a')}p(s_1=s', a_1=a' | s_0=s, a_0=a) E\left[ \sum_{t'=0}^{T} \gamma^{t'} r(s_{t'},a_{t'}) | s_0=s, a_0=a \right]\\
&= r(s,a) + \gamma E[r(s,a)+\gamma Q^{\pi}(s',a')].
\end{align}
$$


2. Bellman's principle of optimality says that an optimal policy $\pi^{*}$ has the following property: whatever the initial state and initial decision are, the remaining decisions must constitute an optimal policy with regard to the state resulting from the first decision. Following this principle, if the optimal value $Q^*(s',a')$ of the sequence $s'$ was known for all possible actions $a'$, then the optimal strategy would be to choose the action $a'$ that maximises the expected value of $r(s,a) + \gamma Q^*(s',a')$, i.e.
$$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].$$


3. So a plausible objective function would be one that minimises the mean square error between $Q(s,a;\theta)$ and $Q^{*}(s,a)$ described above, hence
$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
# The memory is implemented as a circular buffer
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
    
    # Store data
    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[1:]

    # Randomly recover a data from memory
    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # Act via the policy
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])
        #return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))    
        target_q = np.zeros((self.batch_size, 4))
                
        for i in range(self.batch_size):
            # Sample random minibatch of transitions
            [s_, n_s_, a_, r_, game_over_] = self.memory.random_access()
            target_q[i,:] = self.model.predict(np.expand_dims(s_, 0))[0]
            input_states[i] = s_
            
            # Compute target
            if game_over_:
                target_q[i,a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, 0)))
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l
    
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(64,activation ='relu'))
        model.add(Dense(4, activation = "linear"))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))
HTML(display_videos('fc_train49.ogg'))

Epoch 000/050 | Loss 0.0232 | Win/lose count 5.5/3.0 (2.5)
Epoch 001/050 | Loss 0.0248 | Win/lose count 5.5/4.0 (1.5)
Epoch 002/050 | Loss 0.0139 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/050 | Loss 0.0574 | Win/lose count 5.5/6.0 (-0.5)
Epoch 004/050 | Loss 0.0362 | Win/lose count 1.0/9.0 (-8.0)
Epoch 005/050 | Loss 0.0376 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/050 | Loss 0.0079 | Win/lose count 3.0/5.0 (-2.0)
Epoch 007/050 | Loss 0.0306 | Win/lose count 4.0/3.0 (1.0)
Epoch 008/050 | Loss 0.0205 | Win/lose count 2.5/6.0 (-3.5)
Epoch 009/050 | Loss 0.0087 | Win/lose count 3.0/4.0 (-1.0)
Epoch 010/050 | Loss 0.0091 | Win/lose count 3.5/13.0 (-9.5)
Epoch 011/050 | Loss 0.0326 | Win/lose count 8.0/5.0 (3.0)
Epoch 012/050 | Loss 0.0303 | Win/lose count 3.5/2.0 (1.5)
Epoch 013/050 | Loss 0.0284 | Win/lose count 3.0/4.0 (-1.0)
Epoch 014/050 | Loss 0.0070 | Win/lose count 6.0/7.0 (-1.0)
Epoch 015/050 | Loss 0.0264 | Win/lose count 5.5/5.0 (0.5)
Epoch 016/050 | Loss 0.0179 | Win/lose count 1

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [24]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16,(3,3), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Conv2D(32,(3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation = "linear")) 
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [25]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))
HTML(display_videos('cnn_train49.ogg'))

Epoch 000/050 | Loss 0.0187 | Win/lose count 1.5/5.0 (-3.5)
Epoch 001/050 | Loss 0.0011 | Win/lose count 2.0/6.0 (-4.0)
Epoch 002/050 | Loss 0.0023 | Win/lose count 1.0/0 (1.0)
Epoch 003/050 | Loss 0.0151 | Win/lose count 2.0/9.0 (-7.0)
Epoch 004/050 | Loss 0.0180 | Win/lose count 3.0/13.0 (-10.0)
Epoch 005/050 | Loss 0.0209 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/050 | Loss 0.0077 | Win/lose count 5.5/7.0 (-1.5)
Epoch 007/050 | Loss 0.0095 | Win/lose count 3.0/2.0 (1.0)
Epoch 008/050 | Loss 0.0006 | Win/lose count 3.5/4.0 (-0.5)
Epoch 009/050 | Loss 0.0103 | Win/lose count 2.0/3.0 (-1.0)
Epoch 010/050 | Loss 0.0069 | Win/lose count 4.5/4.0 (0.5)
Epoch 011/050 | Loss 0.0104 | Win/lose count 9.0/4.0 (5.0)
Epoch 012/050 | Loss 0.0164 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/050 | Loss 0.0096 | Win/lose count 6.5/0 (6.5)
Epoch 014/050 | Loss 0.0063 | Win/lose count 2.5/6.0 (-3.5)
Epoch 015/050 | Loss 0.0030 | Win/lose count 2.5/3.0 (-0.5)
Epoch 016/050 | Loss 0.0046 | Win/lose count 3.5/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [26]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/2.0. Average score (1.0)
Win/lose count 2.0/2.0. Average score (0.5)
Win/lose count 2.0/4.0. Average score (-0.3333333333333333)
Win/lose count 4.5/9.0. Average score (-1.375)
Win/lose count 4.0/3.0. Average score (-0.9)
Win/lose count 1.0/5.0. Average score (-1.4166666666666667)
Win/lose count 4.5/5.0. Average score (-1.2857142857142858)
Win/lose count 3.5/9.0. Average score (-1.8125)
Win/lose count 2.0/2.0. Average score (-1.6111111111111112)
Win/lose count 2.0/3.0. Average score (-1.55)
Win/lose count 0.5/2.0. Average score (-1.5454545454545454)
Win/lose count 0.5/5.0. Average score (-1.7916666666666667)
Win/lose count 2.0/4.0. Average score (-1.8076923076923077)
Win/lose count 1.5/1.0. Average score (-1.6428571428571428)
Win/lose count 3.5/2.0. Average score (-1.4333333333333333)
Win/lose count 5.5/7.0. Average score (-1.4375)
Win/lose count 0/4.0. Average score (-1.588235294117647)
Win/lose count 2.5/0. Average score (-1.3611111111111112)
Win/los

In [27]:
#HTML(display_videos('cnn_test10.mp4'))
HTML(display_videos('cnn_test19.ogg'))

In [28]:
#HTML(display_videos('fc_test10.mp4'))
HTML(display_videos('fc_test19.ogg'))

The algorithms perform poorly and often present a negative score for temperature T = 0.3. The performance of both algorithms seem to be similar. Nevertheless, they seem to be better than the random agent's one.

We observe that, although the rat starts moving around the map, it gets stuck very easily repeating the same movements. The problem is then that it does not fully explore the map.

By increasing the temperature, the agent seems to move a little more in the beginning, probably because of the abundance of rewards near it.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [47]:
def train_explore(agent,env,epoch,prefix=''):    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # We propose a dynamical choice of the parameter epsilon
            agent.set_epsilon(0.95 agent.epsilon)
            #if lose > win:
            #    agent.set_epsilon((1-e/epoch) * agent.epsilon * 1.5)
            #else:
            #    agent.set_epsilon((1-e/epoch) * agent.epsilon / 1.5)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # Added
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the mouse
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
        
    def draw(self,e):
        # MP4 is not supported by my broser, so I save as ogg as well
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        skvideo.io.vwrite(str(e) + '.ogg', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board > 0,0] = 256
        b[self.board < 0,2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b
        
    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        # Added
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.x,self.y] = 0
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        self.t = 0
    
        # Changed
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [48]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))
HTML(display_videos('cnn_train_explore49.ogg'))

Epoch 000/050 | Loss 0.0083 | Win/lose count 14.0/31.600000000000048 (-17.600000000000048)
Epoch 001/050 | Loss 0.0205 | Win/lose count 10.0/26.70000000000006 (-16.70000000000006)
Epoch 002/050 | Loss 0.0605 | Win/lose count 11.5/31.30000000000008 (-19.80000000000008)
Epoch 003/050 | Loss 0.0090 | Win/lose count 10.5/31.50000000000009 (-21.00000000000009)
Epoch 004/050 | Loss 0.0069 | Win/lose count 11.0/23.70000000000006 (-12.70000000000006)
Epoch 005/050 | Loss 0.0055 | Win/lose count 6.5/29.70000000000009 (-23.20000000000009)
Epoch 006/050 | Loss 0.1131 | Win/lose count 7.0/27.800000000000058 (-20.800000000000058)
Epoch 007/050 | Loss 0.0079 | Win/lose count 7.0/21.200000000000042 (-14.200000000000042)
Epoch 008/050 | Loss 0.0698 | Win/lose count 6.0/30.40000000000005 (-24.40000000000005)
Epoch 009/050 | Loss 0.0043 | Win/lose count 7.5/17.49999999999998 (-9.999999999999979)
Epoch 010/050 | Loss 0.0084 | Win/lose count 7.0/18.29999999999999 (-11.29999999999999)
Epoch 011/050 | Loss 

In [49]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))
HTML(display_videos('cnn_test_explore19.ogg'))

Win/lose count 25.0/11.599999999999973. Average score (13.400000000000027)
Win/lose count 19.5/12.099999999999973. Average score (10.400000000000027)
Win/lose count 25.0/10.19999999999998. Average score (11.86666666666669)
Win/lose count 7.5/17.099999999999973. Average score (6.500000000000025)
Win/lose count 23.0/11.399999999999975. Average score (7.520000000000024)
Win/lose count 20.5/12.299999999999972. Average score (7.633333333333359)
Win/lose count 19.5/14.199999999999966. Average score (7.3000000000000265)
Win/lose count 12.5/14.799999999999963. Average score (6.100000000000028)
Win/lose count 7.5/17.199999999999974. Average score (4.3444444444444725)
Win/lose count 16.5/14.499999999999964. Average score (4.110000000000029)
Win/lose count 29.0/10.19999999999998. Average score (5.445454545454573)
Win/lose count 6.5/19.0. Average score (3.950000000000024)
Win/lose count 25.0/12.79999999999997. Average score (4.584615384615409)
Win/lose count 17.5/13.599999999999968. Average score 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***